In [7]:
DATASET_NAMES = ["les_miserables", "1138_bus", "USpowerGrid"]
PARAMS_NAMES = sorted(["number_of_pivots", "number_of_iterations", "eps"])
COLOR_MAP = {
    DATASET_NAMES[0]: 'red',
    DATASET_NAMES[1]: "green",
    DATASET_NAMES[2]: 'blue'
}
ALPHA = 0.1
print(DATASET_NAMES)
print(PARAMS_NAMES)
print(COLOR_MAP)

['les_miserables', '1138_bus', 'USpowerGrid']
['eps', 'number_of_iterations', 'number_of_pivots']
{'les_miserables': 'red', '1138_bus': 'green', 'USpowerGrid': 'blue'}


In [8]:
# Third Party Library
import pandas as pd

# First Party Library
from config.paths import get_project_root_path


def generate_data_df_dict(dataset_names):
    EXPERIENT_DATA_DIR = (
        get_project_root_path()
        .joinpath("data")
        .joinpath("experiments")
        .joinpath("regression_analysis")
    )

    data_df_dict = {}
    for dataset_name in dataset_names:
        data_path = EXPERIENT_DATA_DIR.joinpath("grid").joinpath(
            f"{dataset_name}-without-pos.pkl"
        )
        data_df_dict[dataset_name] = pd.read_pickle(data_path)

    return data_df_dict


def generate_params_candidates():
    params_steps = {
        "number_of_pivots": 5,
        "number_of_iterations": 10,
        "eps": 0.05,
    }

    params_candidates = {}
    params_name1s = ["number_of_pivots", "number_of_iterations", "eps"]
    for params_name1 in params_name1s:
        params_candidates[params_name1] = [
            v * params_steps[params_name1] for v in list(range(1, 20 + 1))
        ]

    return params_candidates


data_df_dict = generate_data_df_dict(DATASET_NAMES)
params_candidates = generate_params_candidates()


In [9]:
from config.paths import get_project_root_path

def get_model_path(dataset_name, qm_name, pt, p1, p2):
    params_info = f"{p1[0]}={p1[1]}-{p2[0]}={p2[1]}"
    model_path = (
        get_project_root_path()
        .joinpath("data")
        .joinpath("experiments")
        .joinpath("regression_analysis")
        .joinpath("models")
        .joinpath("deg=5-ss")
        .joinpath(f"{dataset_name}-{qm_name}-{pt[0]}-{params_info}.pickle")
    )
    if not model_path.is_file():
        params_info = f"{p2[0]}={p2[1]}-{p1[0]}={p1[1]}"
        model_path = (
            get_project_root_path()
            .joinpath("data")
            .joinpath("experiments")
            .joinpath("regression_analysis")
            .joinpath("models")
            .joinpath("deg=5-ss")
            .joinpath(f"{dataset_name}-{qm_name}-{pt[0]}-{params_info}.pickle")
        )

    return model_path


In [10]:
# parameter（名前＋値）を3つ受け取って、評価値を返す関数

from itertools import combinations
import pickle
import numpy as np


def regression_func(ps, qm_name):
    qs = []
    cs = list(combinations(ps, 2))

    for c in cs:
        p1 = c[0]
        p2 = c[1]
        pt = list(
            filter(
                lambda x: x[0] != p1[0] and x[0] != p2[0],
                ps,
            )
        )[0]

        for dataset_name in DATASET_NAMES:
            model_path = get_model_path(
                dataset_name=dataset_name, qm_name=qm_name, pt=pt, p1=p1, p2=p2
            )
            with open(model_path, mode="rb") as f:
                regr = pickle.load(f)

            q = regr.predict(np.array([pt[1]]).reshape(-1, 1))[0][0]
            qs.append(q)

    result = sum(qs) / len(qs)
    return result


In [11]:
regression_func(
    [
        ["eps", 0.1],
        ["number_of_iterations", 100],
        ["number_of_pivots", 50],
    ],
    "stress",
)


-0.30687576391745647

In [12]:
from itertools import combinations
import pickle
from config.paths import get_project_root_path
from config.quality_metrics import ALL_QM_NAMES
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import uuid

cs = list(combinations(PARAMS_NAMES, 2))

for c in cs:
    px = c[0]
    py = c[1]

    pe = list(filter(lambda x: x != px and x != py, PARAMS_NAMES))[0]
    pxvs = params_candidates[px]
    pyvs = params_candidates[py]
    pevs = params_candidates[pe]
    for qm_name in ALL_QM_NAMES:
        for pev in pevs:
            fig = plt.figure(figsize=(8, 8))
            axis = fig.add_subplot(111, projection="3d")
            fig.subplots_adjust(right=1, top=0.93)
            axis.set_title(f"{qm_name}", fontsize=28)
            axis.set_xlabel(px, fontsize=20)
            axis.set_ylabel(py, fontsize=20)
            axis.invert_xaxis()
            df_data = []

            for pyv in pyvs:
                for pxv in pxvs:
                    q = regression_func(
                        ps=[[pe, pev], [px, pxv], [py, pyv]], qm_name=qm_name
                    )
                    df_data.append(
                        {
                            "index": uuid.uuid4(),
                            f"{pe}": pev,
                            f"{px}": round(pxv, 2),
                            f"{py}": pyv,
                            "q": q,
                        }
                    )

            # df_heatmap = pd.pivot_table(
            #     data=pd.DataFrame(df_data), values="q", columns=px, index=py
            # )
            # sns.heatmap(df_heatmap, cmap='RdBu')
            # axis.invert_yaxis()
            # axis.set_xticklabels(axis.get_xticklabels(), rotation=-45)
            df = pd.DataFrame(df_data)
            # print((df[px], df[py], df['q']))
            for pyv in pyvs:
                pdf = df.query(f"{py} == {pyv} & {pe} == {pev}")
                axis.plot(pdf[px], pdf[py], pdf["q"])
            for pxv in pxvs:
                pdf = df.query(f"{px} == {pxv} & {pe} == {pev}")
                axis.plot(pdf[px], pdf[py], pdf["q"])
            export_path = (
                get_project_root_path()
                .joinpath(
                    "data/experiments/regression_analysis/regression_function_heatmap"
                )
                .joinpath("tmp")
                .joinpath(f"{qm_name}-{px}-{py}-{pe}={pev}-plot.png")
            )
            export_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(export_path, dpi=300)
            plt.close()


In [13]:
from PIL import Image, ImageDraw, ImageFont


def get_concat_h(im1, im2):
    dst = Image.new("RGB", (im1.width + im2.width, im1.height), "black")
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2):
    dst = Image.new("RGB", (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [14]:
from itertools import combinations
import pickle
from config.paths import get_project_root_path
from config.quality_metrics import ALL_QM_NAMES
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import uuid

cs = list(combinations(PARAMS_NAMES, 2))

for c in cs:
    px = c[0]
    py = c[1]

    pe = list(filter(lambda x: x != px and x != py, PARAMS_NAMES))[0]
    pxvs = params_candidates[px]
    pyvs = params_candidates[py]
    pevs = params_candidates[pe]
    for pev in pevs:
        images = []
        tmp = []
        for qm_name in ALL_QM_NAMES:
            img_path = (
                get_project_root_path()
                .joinpath(
                    "data/experiments/regression_analysis/regression_function_heatmap"
                )
                .joinpath("tmp")
                .joinpath(f"{qm_name}-{px}-{py}-{pe}={pev}-plot.png")
            )
            img = Image.open(img_path)

            tmp.append({"image": img})

            if len(tmp) == 3:
                images.append(tmp)
                tmp = []

        dst = None
        for v in images:
            h_dst = None
            for h in v:
                if h_dst is None:
                    h_dst = h["image"]
                    continue
                h_dst = get_concat_h(h_dst, h["image"])
            if dst is None:
                dst = h_dst
                continue
            dst = get_concat_v(dst, h_dst)
        dst = add_margin(dst, 220, 0, 0, 0, '#fff')
        draw = ImageDraw.Draw(dst)
        font = ImageFont.truetype("Arial.ttf", 200)
        draw.text((40, 40), f"{pe}={pev}", "red", font=font)
        dst.save(
            (
                get_project_root_path()
                .joinpath(
                    "data/experiments/regression_analysis/regression_function_heatmap"
                )
                .joinpath(f"{px}-{py}-{pe}={pev}-plot.png")
            )
        )
